### **Random Forests** 

Random Forest is a **bagging** technique that **trains multiple decision trees** with **minor modification** in split criterion.

In case of decision trees, we train a single decision tree. 

In random forest we train multiple decision trees on different training sets obtained through bagging (aka Boostrap Agreggation).

### **Algorithm**
**Input**:

1. The training data $D$ with shape $(n,m)$, say $D_1,D_2,\ldots ,D_q$ with replacement from $D$.

2. In each of the datasets $D_j$, select $u$ out of $m$ where $u \le m$ features before each split and train a full decision tree $h_j(\mathbf x)$

3. The final predictor : 

  * For regression, an average output from $q$ regressors is assigned to the new example:
  
  $$ h(\mathbf x) = \frac {1}{q} \sum_{j=1}^q h_j(\mathbf x) $$

  * For classification, a majority voting is taken and the class label with the maximum number of votes is assigned to the new example.

### **Implementation**

In order to keep the implementation focussed to main components of random forest, we make use of `DecisionTreeClassifier` from scikit-learn library for the decision tree components.

In [40]:
import numpy as np
np.random.RandomState(1)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
from sklearn.tree import DecisionTreeClassifier

#### **Bagging**

We define a function for bagging - creating boostrap samples $D_1,D_2,\ldots,D_q$ from the original dataset $D:$

The **key step** is : 

`np.random.choice` with **size=n_samples** and **replace=True** 

It ensures that the boostrapped sample has the same number of samples as the original dataset and it is obtained by sampling with replacement.

In [41]:
def bag(X, y):
    # Count rows
    n_samples = X.shape[0]

    # generate samples from input
    indices = np.random.choice(n_samples, size=n_samples,
                               replace=True)
    return X[indices], y[indices]

In [42]:
# n_samples = 5
#     # generate samples from input
# indices = np.random.choice(n_samples, size = n_samples,
#                             replace=True)
# indices

**Majority Voting**

Count most common label to obtain majority vote for class labels.

In [43]:
y = [1, 1, 1, 0, 0, 2, 2, 2, 2, 3, 3, 3, 3, 3]
print(Counter(y))
print(Counter(y).most_common())

Counter({3: 5, 2: 4, 1: 3, 0: 2})
[(3, 5), (2, 4), (1, 3), (0, 2)]


In [44]:
# most_common returns n most frequent elem count
Counter(y).most_common(1)[0]

(3, 5)

In [45]:
Counter(y).most_common()[1][0]

2

In [46]:
def most_comon_label(y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common

**Random forest class**

We create a `RandomForest` class with the following default parameters :

* number of trees = 10

* minimum number of samples = 2

* maximum depth = 100 

The `max_features` is a configurable parameter that can be set by the user.

In [47]:
class RandomForest:
    def __init__(self, n_trees = 10, min_sample_split = 2,
                 max_depth = 100, max_features = None):
        
        # hpt for fixing number of trees
        self.n_trees = n_trees

        # max depth
        self.max_depth = max_depth

        # min sample split
        self.min_sample_split = min_sample_split

        # max features
        self.max_features = max_features
        self.trees = []


**Training Random Forest**
We implement the `fit` method.

* Initialize an empty list of decision tree classifiers.

* In the `for` loop, we train each decision tree with parameters set from random forest on a boostrapped sample obtained through the `bag` function.

In [48]:
def fit(self, X, y):
    # empty array to store trees
    self.trees = []

    for _ in range(self.n_trees):
        tree = DecisionTreeClassifier(
            min_samples_split=self.min_samples_split,
            max_depth=self.max_depth,
            max_features = self.max_features)
        
        X_sample, y_sample = bag(X, y)
        tree.fit(X_sample, y_sample)
        # append each of trees in list
        self.trees.append(tree)

**Inference**
Let's implement `predict` function : 

Here, we need to note that each of the trees will give predictions for all the individual rows of the input data.

**For example :** If we have a random forest with 3 trees and 2 classes 0 & 1, let's assume the prediciton for 5 samples is as follows : 

* Tree 1 gives 00111

* Tree 2 gives 11001

* Tree 3 gives 10101

We need to aggregate the output for the respective samples and take an average / majority vote. For this, we will use `np.swapaxes`.

In [49]:
def predict(self, X):
    tree_predict = np.array([tree.predict(X) for tree in self.trees])

    # each tree will give predictions
    tree_predict = np.swapaxes(tree_predict, 0, 1)

    # get most common lsbel for each class
    y_pred = [most_comon_label(tree_pred) for tree_pred in tree_predict]
    return np.array(y_pred)

**Combine all components**

In [50]:
def bag(X, y):
    n_samples = X.shape[0]
    indices = np.random.choice(n_samples, size=n_samples, replace=True)
    return X[indices], y[indices]


def most_common_label(y):
  counter = Counter(y)
  most_common = counter.most_common(1)[0][0]
  return most_common


class RandomForest:
  def __init__(self, n_trees=10, min_samples_split=2, max_depth=100, max_features=None):
    self.n_trees = n_trees  
    self.min_samples_split = min_samples_split
    self.max_depth = max_depth  
    self.max_features = max_features 
    self.trees = []

  def fit(self, X, y):
    self.trees = []

    for _ in range(self.n_trees):
      tree = DecisionTreeClassifier(  
          min_samples_split=self.min_samples_split,
          max_depth=self.max_depth,
          max_features=self.max_features)

      X_sample, y_sample = bag(X, y)
      tree.fit(X_sample, y_sample)
      self.trees.append(tree)  

  def predict(self, X):
    tree_predict = np.array([tree.predict(X) for tree in self.trees])
    tree_predict = np.swapaxes(tree_predict, 0, 1)

    y_pred = [most_common_label(tree_pred) for tree_pred in tree_predict]
    return np.array(y_pred)